In [16]:
import re
def rulematch():
	rulefilename = "./misuseprofile"
	print(rulefilename);
	rulefd = open(rulefilename,"r");
	print("----------------");
	funcnameset = set();
	funcdetail = dict();
	wfuncdetail = dict();
	ruledesresult = dict();
	wfuncset=set();
	ruledesflag = False;
	misusefunctionflag = False;
	dependflag = False;
	while True:
		line = rulefd.readline().strip();
		if not line:
			break;
		if line == "ruledes:":
			ruledesflag = True;
			misusefunctionflag = False;
			wrongflag = False;
		elif line == "misusefunction:":
			ruledesflag = False;
			misusefunctionflag = True;
			wrongflag = False;
		elif line == "dependfunc:":
			wrongflag = True;
			ruledesflag = False;
			dependflag = False;

		if misusefunctionflag:
			if not len(line) == 0 and line[0] == "#":
				continue;
			field = line.split("\t");
			if not len(field) == 2:
				continue;
			funcprototype = re.split("[(),]",field[0].strip());
			if len(funcprototype) == 0:
				continue;
			funcheader = funcprototype[0].split(" ");
			if not len(funcheader) == 2:
				continue;
			if funcheader[1][0] == "*":
				funcheader[1] = funcheader[1][1:len(funcheader[1])];
				funcheader[0] += "*";
			funcnameset.add(funcheader[1]);
			tmpfunction=function();
			tmpfunction.setfunctionname(funcheader[1]);
			tmpfunction.setreturntype(funcheader[0]);
			for i in range(1,len(funcprototype)):
				if funcprototype[i] == "":
					continue;
				tmpfunction.addarg(funcprototype[i].strip());
			farglocat = field[1].split(";");
			for glocatg in farglocat:
				glocat = glocatg.split(":");
				if not len(glocat) == 4:
					continue;
				if glocat[1].strip() == "string":
					tmpfunction.addfarg([int(glocat[0]),FARGTYPE.Vstring]);
				elif glocat[1].strip() == "int":
					tmpfunction.addfarg([int(glocat[0]),FARGTYPE.Vint]);
				elif glocat[1].strip() == "len":
					tmpfunction.addfarg([int(glocat[0]),FARGTYPE.Vlen]);
				elif glocat[1].strip() == "none":
					tmpfunction.addfarg([int(glocat[0]),FARGTYPE.Vnone]);
				tmpfunction.addrule(int(glocat[2]));
				tmpfunction.addruledes(ruledesresult[int(glocat[2])]);
				tmpfunction.addlimit(glocat[3]);
			funcdetail[funcheader[1]] = tmpfunction;
		elif ruledesflag:
			if not len(line) == 0 and line[0] == "#":
				continue;
			field = line.split("\t");
			if not len(field) == 2:
				continue;
			ruledesresult[int(field[0])] = field[1];
		elif dependflag:
			if not len(line) == 0 and line[0] == "#":
				continue;
			functionfield = line.split("->");
			if not len(field) == 2:
				continue;
			funcprototype = re.split("[(),]",field[0].strip());
			if len(funcprototype) == 0:
				continue;
			funcheader = funcprototype[0].split(" ");
			if not len(funcheader) == 2:
				continue;
			if funcheader[1][0] == "*":
				funcheader[1] = funcheader[1][1:len(funcheader[1])];
				funcheader[0] += "*";
			wfuncset.add(funcheader[1]);
			tmpfunction=function();
			tmpfunction.setfunctionname(funcheader[1]);
			tmpfunction.setreturntype(funcheader[0]);
			for i in range(1,len(funcprototype)):
				if funcprototype[i] == "":
					continue;
				tmpfunction.addarg(funcprototype[i].strip());
			farglocat = field[1].split(";");
			for glocatg in farglocat:
				glocat = glocatg.split(":");
				if not len(glocat) == 4:
					continue;
				if glocat[1].strip() == "string":
					tmpfunction.addfarg([int(glocat[0]),FARGTYPE.Vstring]);
				elif glocat[1].strip() == "int":
					tmpfunction.addfarg([int(glocat[0]),FARGTYPE.Vint]);
				tmpfunction.addrule(int(glocat[2]));
				tmpfunction.addruledes(ruledesresult[int(glocat[2])]);
				tmpfunction.addlimit(glocat[3]);
			wfuncdetail[funcheader[1]] = tmpfunction;
			
	rulefd.close();
	return funcnameset,funcdetail,ruledesresult,wfuncset,wfuncdetail;

In [17]:
class function:
	def __init__(self):
		self.functionname = "";
		self.addr = list();
		self.arg = list();
		self.farg = list();
		self.aargloffset = list();
		self.rule=list();
		self.ruledes=list();
		self.limitl=list();
		self.functiontype = "";
		self.returntype = "";

	def addlimit(self,limit):
		self.limitl.append(limit);

	def getlimitindex(self,index):
		return self.limitl[index];

	def getrule(self):
		return self.rule;

	def setrule(self,rule):
		self.rule = rule;

	def addruledes(self,ruledesitem):
		self.ruledes.append(ruledesitem);

	def addrule(self,rulel):
		self.rule.append(rulel);

	def getruleindex(self,index):
		return self.rule[index];
	
	def getruledesindex(self,index):
		return self.ruledes[index];

	def getfunctionname(self):
		return self.functionname;

	def setfunctionname(self,functionname):
		self.functionname = functionname;

	def getfunctiontype(self):
		return self.functiontype;

	def setfunctiontype(self,functiontype):
		self.functiontype = functiontype;

	def getarg(self):
		return self.arg;

	def addarg(self,arg):
		self.arg.append(arg);

	def fillarg(self,argitem,index):
		while len(self.arg) <= index:
			self.arg.append("");
		self.arg[index] = argitem;
	
	def addfarg(self,arg):
		self.farg.append(arg);

	def getfarg(self):
		return self.farg;
	
	def farglen(self):
		return len(self.farg);

	def getfargitem(self,index):
		return self.farg[index];
	
	def setreturntype(self,returntype):
		self.returntype = returntype;
	
	def getreturntype(self):
		return self.returntype;

	def getfunctionaddr(self):
		return self.addr;

	def addfunctionaddr(self,addr):
		if not addr in self.addr:
			self.addr.append(addr);

	def clearfunctionaarg(self):
		self.aargloffset = list();

	def addfunctionaarg(self,offset):
		self.aargloffset.append(offset);

	def getfunctionaarg(self,locat):
		if len(self.aargloffset) <= locat:
			return None;
		return self.aargloffset[locat];

	def setfunctionaargoffset(self,locat,v):
		if len(self.aargloffset) <= locat:
			return False;
		self.aargloffset[locat] = v;
		return True;
	
	def getfunctionaarglist(self):
		return self.aargloffset;

	def listallfagrpp(self):
		print(self.functionname);
		'''for i in range(0,len(self.addr)):
			msg = "";
			msg += "addr:0x%X "%self.addr[i];
			print(msg);'''
		for fargitemindex in range(0,len(self.farg)):
			msg = "";
			fargitem = self.farg[fargitemindex];
			faddrv = fargitem[0];
			faddrtype = fargitem[1];
			msg += " faddr:%d"%faddrv;
			msg += " faddrtype:%d"%faddrtype;
			msg += " limit:%s"%self.limitl[fargitemindex];
			msg += " rule:%d"%self.rule[fargitemindex];
			msg += " ruledes:%s"%self.ruledes[fargitemindex];
			#msg += ""
			print(msg);
            
class FARGTYPE:
	Vstring = 1,
	Vint = 2,
	Vlen = 3,
	Vnone = 4

In [18]:
funcnameset,funcdetail,ruledesresult,wfuncset,wfuncdetail = rulematch()

./misuseprofile
----------------


In [31]:
funcdetail['srand'].getrule()

[5]